In [1]:
import sys
import pandas as pd
sys.path.insert(0, '../src/')

In [2]:
from data.data_load import DataLoad
from data.data_validation import DataValidation

In [3]:
dl = DataLoad()
dv = DataValidation()

In [4]:
df = dl.load_data('train_dataset_name')

2023-12-02 19:57:58 [info     ] Iniciando o carregamento


In [5]:
df

,target,TaxaDeUtilizacaoDeLinhasNaoGarantidas,Idade,NumeroDeVezes30-59DiasAtrasoNaoPior,TaxaDeEndividamento,RendaMensal,NumeroDeLinhasDeCreditoEEmprestimosAbertos,NumeroDeVezes90DiasAtraso,NumeroDeEmprestimosOuLinhasImobiliarias,NumeroDeVezes60-89DiasAtrasoNaoPior,NumeroDeDependentes
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
149995,0,0.040674,74,0,0.225131,2100.0,4,0,1,0,0.0
149996,0,0.299745,44,0,0.716562,5584.0,4,0,1,0,2.0
149997,0,0.246044,58,0,3870.000000,NaN,18,0,1,0,0.0
149998,0,0.000000,30,0,0.000000,5716.0,4,0,0,0,0.0


In [6]:
is_valid = dv.run(df)

2023-12-02 19:57:58 [info     ] Validação iniciou..
2023-12-02 19:57:58 [info     ] Validation columns passed...
2023-12-02 19:57:58 [info     ] Validacao com sucesso.


In [7]:
from data.data_transformation import DataTransformation

In [8]:
dt = DataTransformation(df)

In [9]:
X_train, X_valid, y_train, y_valid = dt.train_test_split()

In [10]:
X_train.shape

(100500, 10)

In [11]:
X_valid.shape

(49500, 10)

In [12]:
from sklearn.pipeline import Pipeline
from feature_engine.discretisation import EqualFrequencyDiscretiser, EqualWidthDiscretiser
from feature_engine.imputation import MeanMedianImputer
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.preprocessing import RobustScaler, StandardScaler

In [13]:
from data.data_preprocess import DataPreprocess
from utils.utils import load_config_file

In [14]:
pipe = Pipeline([
    ('imputer', MeanMedianImputer(variables=load_config_file().get('vars_imputer'))),
    ('discretizer', EqualFrequencyDiscretiser(variables=load_config_file().get('vars_discretizer'))),
    ('scaler', SklearnTransformerWrapper(StandardScaler()))
])

In [15]:
dp = DataPreprocess(pipe)

In [16]:
dp.train(X_train)

2023-12-02 19:57:59 [info     ] Iniciando o processamento


Pipeline(steps=[('imputer',
                 MeanMedianImputer(variables=['RendaMensal',
                                              'NumeroDeDependentes'])),
                ('discretizer',
                 EqualFrequencyDiscretiser(variables=['TaxaDeUtilizacaoDeLinhasNaoGarantidas',
                                                      'TaxaDeEndividamento',
                                                      'RendaMensal'])),
                ('scaler',
                 SklearnTransformerWrapper(transformer=StandardScaler()))])

In [17]:
X_train_processed = dp.transform(X_train)
X_valid_processed = dp.transform(X_valid)

2023-12-02 19:57:59 [info     ] Iniciando a Transformação
2023-12-02 19:57:59 [info     ] Iniciando a Transformação


In [18]:
X_train_processed.shape

(100500, 10)

In [19]:
X_valid_processed.shape

(49500, 10)

In [20]:
from data.train_model import TrainModel

In [21]:
y_train = pd.DataFrame(y_train)

In [22]:
tm = TrainModel(dados_x=X_train_processed, dados_y=y_train)

In [23]:
from sklearn.linear_model import LogisticRegression

In [24]:
algoritmo = LogisticRegression()
model = tm.train(model=algoritmo)

c:\Users\thale\Documents\Projetos_DS\ml_flow\evn\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
